**IMPORTS**

In [6]:
from os.path import exists
import pandas as pd

df__path = 'data/data/train.csv'
df=pd.read_csv(df__path)
df=df.sample(n=19_000,random_state=0)
df["store"]=df["store"].astype(str)
df["item"]=df["item"].astype(str)

df["date"]=pd.to_datetime(df["date"])

df.sort_values("date",inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [8]:
#training range
train=df[df['date']<='2017-1-1']
test=df[df['date']>'2017-1-1']

In [ ]:
train_features=train.drop(columns=["sales"])
train_target=train["sales"]
test_features=test.drop(columns=["sales"])
test_target=test["sales"]

In [10]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher=FeaturesEnricher(
    search_keys={"date":SearchKey.DATE,},
    cv=CVType.time_series
)
enricher.fit(
    train_features,train_target,eval_set=[(test_features,test_target)]
)

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
target,All valid,-
date,All valid,-




Running search request, search_id=93acfd39-aaf4-4d48-bc7a-0cf38722d6a9
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
-

10 relevant feature(s) found with the search keys: ['date']


item,12.3308,100.0000,"50, 43, 35",,,
store,8.0479,100.0000,"5, 3, 7",,,
f_events_date_week_sin1_847b5db1,3.8859,100.0000,"0.7818, -0.9749, -0.7818",Upgini,Calendar data,Daily
f_financial_date_crude_oil_7d_to_1y_c3e0ad17,2.8190,100.0000,"1.0109, 1.0702, 1.0146",Upgini,Markets data,Daily
f_autofe_lag_1d_f7664f8ba0,2.6546,99.8623,"0.3416, -0.263, -0.3817","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_events_date_year_cos1_9014a856,2.3941,100.0000,"0.3416, -0.263, -0.3817",Upgini,Calendar data,Daily
f_events_date_week_cos3_7525fe31,2.0072,100.0000,"-0.901, 0.6235, -0.901",Upgini,Calendar data,Daily
f_autofe_delta_30d_929f2f9bc7,0.8089,96.8757,"-5.0, -69.0, -28.0",Training dataset,AutoFE: features from Training dataset,
f_economic_date_cbpol_umap_6_aa0352de,0.5778,100.0000,"1.0513, 6.9379, 1.0491",Upgini,World economic indicators,Daily
f_events_date_year_cos2_7acbaf1b,0.2701,100.0000,"-0.7667, -0.9473, -0.819",Upgini,Calendar data,Daily


Upgini,Calendar data,8.5573,4
Upgini,Markets data,2.8190,1
"Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",2.6546,1
Training dataset,AutoFE: features from Training dataset,0.8089,1
Upgini,World economic indicators,0.5778,1


"Training dataset,Calendar data",f_autofe_lag_1d_f7664f8ba0,f_events_date_year_cos1_9014a856,lag_1d
Training dataset,f_autofe_delta_30d_929f2f9bc7,target,delta_30d


We detected 48 outliers in your sample.
Examples of outliers with maximum value of target:
33    205
17    196
12    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


Train,9417,53.3341,0.322 ± 0.104,0.289 ± 0.078,0.0330,10.4%
Eval 1,3752,58.6493,0.278 ± 0.007,0.266 ± 0.022,0.0120,4.4%


**REGRESSION MODEL**

In [12]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model=CatBoostRegressor(
    verbose=False,
    allow_writing_files=False,
    random_state=0
)
enricher.calculate_metrics(
    train_features,train_target,
    eval_set=[(test_features,test_target)],
    estimator=model,
    scoring="mean_absolute_error"

)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_error,Enriched mean_absolute_error,"Uplift, abs","Uplift, %"
0,Train,9417,53.3341,12.967 ± 1.905,8.681 ± 1.258,4.286,33.1%
1,Eval 1,3752,58.6493,13.560 ± 0.598,10.247 ± 0.782,3.312,24.4%


In [15]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)

# Ensure the result is a DataFrame before calling head()
if enriched_train_features is not None:
	enriched_train_features.head()
else:
	print("enriched_train_features is None. Please check the enricher configuration.")


[                                                            ] 0% Checking dataset...

You are trying to launch enrichment for 15225 rows, which will exceed the rest limit 1000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[============================================================] 100% Finished

You are trying to launch enrichment for 3775 rows, which will exceed the rest limit 1000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

enriched_train_features is None. Please check the enricher configuration.
